In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("abraxas")

create_database('blp_dataset')
create_table('abraxas_tmp')

In [ ]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\listing")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())
            
            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                desc_content = page_content.find("div", attrs={"class": "col-sm-6"})

                if desc_content is not None:
                    raw_desc = desc_content.find("div")

                    description = get_content(raw_desc)

                ## GET TITLE
                title = ""
                raw_title = page_content.find("h2")

                title = get_content(raw_title)

                ## GET SELLER
                seller_name = ""

                if desc_content is not None:
                    raw_seller = desc_content.find("a")

                    seller_name = get_content(raw_seller)

                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                    values = (title, description, "Abraxas", seller_name, "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    raw_name = page_content.find("img")
                    name = ""

                    if raw_name is not None:
                        try:
                            name = raw_name["src"]
                        except IndexError:
                            print(f"Image {product_id} has no src")
                            update_image(product_id, "no_image")

                    if name is not "":
                        url_image = Path(f"{folder_image}\{name}")

                        try:
                            # copy images to new directory, renaming with product id
                            if url_image.exists():
                                print(f"Image {product_id} exists... creating")

                                dst_dir = Path(f"D:\images\p\{product_id}.jpg")
                                copyfile(url_image, dst_dir)
                            else:
                                print(f"Image {product_id} does not exists")
                                update_image(product_id, "no_image")
                        except:
                            print(f"Image {product_id} has error. Except")
                            update_image(product_id, "no_image")
                    else:
                        print(f"Image {product_id} has no name")
                        update_image(product_id, "no_image")